# Libraries 

In [1]:
import nltk
#to open csv file
import csv
import pandas as pd
import numpy as np
#sentences & words tokenization
from nltk.tokenize import sent_tokenize, word_tokenize
#regular expression 
import re
#for stopwords
from nltk.corpus import stopwords
import string
# from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.stem import ISRIStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
#for preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hends\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hends\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hends\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# csv path

In [3]:
dataset_path = r'C:\Users\hends\Documents\ANLP\dataset\arabic_english.csv'
dataset = pd.read_csv(dataset_path, encoding="utf-8")
dataset.head()

,arabic,english
0,متى أنشئت هذه الجامعة؟,When was this university founded?
1,أراها نادراً,I see it rarely.
2,يعزف على البيانو بشكل جيد جداً,He plays the piano very well.
3,مع كل احترامي.,With all due respect.
4,نظف أسنانك,Brush your teeth clean.


# preprocessing function

### arabic preprocessing

In [4]:
def preprocess_text(text):
    # Apply lowercase
    text = text.lower()
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove Arabic stopwords
    stop_words = set(stopwords.words('arabic'))
    tokens = [token for token in tokens if token not in stop_words]
    # Remove Arabic punctuation and other non-alphanumeric characters
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    # Remove empty tokens
    tokens = [token for token in tokens if token]
    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Apply stemming
    stemmer = ISRIStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

In [5]:
dataset['arabic_preprocessed'] = dataset['arabic'].apply(preprocess_text)

In [6]:
dataset['arabic_preprocessed']

0                       [شئت, جمع]
1                       [ارا, ندر]
2        [عزف, ينو, شكل, جيد, جدا]
3                            [حرم]
4                       [نظف, سنن]
                   ...            
34884    [ذهب, انت, يسر, ونأ, يمي]
34885         [يجب, قرء, لنص, بتأ]
34886                   [عند, شكل]
34887                   [عند, شكل]
34888                   [عند, شكل]
Name: arabic_preprocessed, Length: 34889, dtype: object

### english preprocessing

In [7]:
def preprocess_english_text(text):
    # Apply lowercase
    text = text.lower()
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove English stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Remove English punctuation and other non-alphanumeric characters
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    # Remove empty tokens
    tokens = [token for token in tokens if token]
    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

In [8]:
dataset['english_preprocessed'] = dataset['english'].apply(preprocess_english_text)

In [9]:
dataset['english_preprocessed']

0                  [university, founded]
1                          [see, rarely]
2                    [play, piano, well]
3                         [due, respect]
4                  [brush, teeth, clean]
                      ...               
34884              [go, left, go, right]
34885    [must, read, textbook, closely]
34886                          [problem]
34887                 [ve, got, problem]
34888                 [ve, got, problem]
Name: english_preprocessed, Length: 34889, dtype: object

# Feature Extraction

### word embedding

In [10]:
from gensim.models import Word2Vec

# Assuming you have lists of tokenized Arabic and English sentences
arabic_corpus = [sentence for sentence in dataset['arabic_preprocessed']]
english_corpus = [sentence for sentence in dataset['english_preprocessed']]

In [11]:
# Train Arabic Word2Vec model
arabic_model = Word2Vec(sentences=arabic_corpus, vector_size=300, window=5, min_count=5, workers=4)

# Train English Word2Vec model
english_model = Word2Vec(sentences=english_corpus, vector_size=300, window=5, min_count=5, workers=4)

In [12]:
arabic_model.save('arabic_word2vec.model')
english_model.save('english_word2vec.model')

In [13]:
from gensim.models import Word2Vec

arabic_model = Word2Vec.load('arabic_word2vec.model')
english_model = Word2Vec.load('english_word2vec.model')

In [14]:
import numpy as np

def text_to_embeddings(text, model):
    embeddings = []
    for word in text:
        if word in model.wv:
            embeddings.append(model.wv[word])
        else:
            # Handle out-of-vocabulary words
            embeddings.append(np.zeros(model.vector_size))
    return np.array(embeddings)

In [15]:
dataset['arabic_embeddings'] = dataset['arabic_preprocessed'].apply(lambda x: text_to_embeddings(x, arabic_model))
dataset['english_embeddings'] = dataset['english_preprocessed'].apply(lambda x: text_to_embeddings(x, english_model))

In [16]:
dataset['arabic_embeddings']

0        [[0.012007201, 0.097810835, 0.02885719, 0.0256...
1        [[0.006580001, 0.06165389, 0.021370022, 0.0136...
2        [[0.036001354, 0.2164926, 0.066832535, 0.05173...
3        [[0.030880434, 0.22037748, 0.06586347, 0.05477...
4        [[0.029636184, 0.23839317, 0.0652243, 0.059855...
                               ...                        
34884    [[0.03176287189126015, 0.27656808495521545, 0....
34885    [[0.038611628115177155, 0.28334930539131165, 0...
34886    [[0.04234245, 0.29211202, 0.08000256, 0.069772...
34887    [[0.04234245, 0.29211202, 0.08000256, 0.069772...
34888    [[0.04234245, 0.29211202, 0.08000256, 0.069772...
Name: arabic_embeddings, Length: 34889, dtype: object

In [17]:
dataset['english_embeddings']

0        [[0.023564283, 0.21296181, -0.009773556, 0.106...
1        [[0.017972827, 0.23550971, -0.0066297464, 0.11...
2        [[0.027597813, 0.2323528, -0.018253095, 0.1107...
3        [[0.010202153, 0.09688939, -0.0017658105, 0.05...
4        [[0.010366553, 0.095281266, -0.0074263937, 0.0...
                               ...                        
34884    [[0.021633, 0.23766693, -0.022877734, 0.128194...
34885    [[0.029085808, 0.25794262, -0.0073842155, 0.13...
34886    [[0.022605492, 0.24313901, -0.0070547853, 0.12...
34887    [[0.029859865, 0.24789649, -0.020183185, 0.137...
34888    [[0.029859865, 0.24789649, -0.020183185, 0.137...
Name: english_embeddings, Length: 34889, dtype: object

# Model architecture

### LSTM (seq2seq)

In [18]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [19]:
# Define the maximum sequence length for both input and output sequences
max_encoder_seq_length = max(len(seq) for seq in dataset['arabic_preprocessed'])
max_decoder_seq_length = max(len(seq) for seq in dataset['english_preprocessed'])

# Define the input sequence
encoder_inputs = Input(shape=(max_encoder_seq_length, arabic_model.vector_size))

# Define the LSTM encoder
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the input sequence for the decoder
decoder_inputs = Input(shape=(max_decoder_seq_length, english_model.vector_size))

# Define the LSTM decoder
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Define the Dense layer for output
decoder_dense = Dense(len(english_model.wv.key_to_index), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Print the model summary
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 104, 300)          │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 89, 300)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 256), (None,      │         570,368 │ input_layer[0][0]          │
│                               │ 256), (None, 256)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, 89, 256), (None,  │         570,368 │ input_layer_1[0][0],       │
│                               │ 256), (None, 256)]        │                 │ lstm[0][1], lstm[0][2]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 89, 2646)          │         680,022 │ lstm_1[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,820,758 (6.95 MB)

 Trainable params: 1,820,758 (6.95 MB)

 Non-trainable params: 0 (0.00 B)

None


# Data Preparation

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load your dataset (assumes a CSV file with 'arabic' and 'english' columns)
df = pd.read_csv("arabic_english_dataset.csv")  # Replace with your dataset filename
df = df.dropna()

# Optional: Add <start> and <end> tokens to target (English) sentences
df['english'] = df['english'].apply(lambda x: '<start> ' + x + ' <end>')

# Split data
input_texts = df['arabic'].values
target_texts = df['english'].values

# Tokenize Arabic
arabic_tokenizer = Tokenizer()
arabic_tokenizer.fit_on_texts(input_texts)
input_sequences = arabic_tokenizer.texts_to_sequences(input_texts)
input_sequences = pad_sequences(input_sequences, padding='post')

# Tokenize English
english_tokenizer = Tokenizer(filters='')
english_tokenizer.fit_on_texts(target_texts)
target_sequences = english_tokenizer.texts_to_sequences(target_texts)
target_sequences = pad_sequences(target_sequences, padding='post')

# Vocabulary sizes
input_vocab_size = len(arabic_tokenizer.word_index) + 1
target_vocab_size = len(english_tokenizer.word_index) + 1

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_sequences, target_sequences, test_size=0.2)


ModuleNotFoundError: No module named 'numpy'

#  Model Training

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

embedding_dim = 256
lstm_units = 512

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Shift y_train by 1 timestep for decoder target
y_train_shifted = np.expand_dims(y_train[:, 1:], -1)
y_val_shifted = np.expand_dims(y_val[:, 1:], -1)

# Train
model.fit(
    [X_train, y_train[:, :-1]],
    y_train_shifted,
    validation_data=([X_val, y_val[:, :-1]], np.expand_dims(y_val[:, 1:], -1)),
    batch_size=64,
    epochs=10
)
